<a href="https://colab.research.google.com/github/Kopyshevskiy/AN2DL_ch2/blob/main/unet2_with_resconnections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from google.colab import drive
drive.mount("/gdrive")
%cd /gdrive/My Drive/AN2DL/Homework2

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/AN2DL/Homework2


In [27]:
# Set seed for reproducibility
seed = 42

# Import necessary libraries
import os

# Set environment variables before importing modules
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(seed)
random.seed(seed)

# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Set seed for TensorFlow
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Reduce TensorFlow verbosity
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Print TensorFlow version
print(tf.__version__)

# Import other libraries
import os
import math
from PIL import Image
from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors

from datetime import datetime
import pandas as pd

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

2.17.1


In [46]:
# load the data
data = np.load("mars_for_students.npz")

training_set = data["training_set"]
X_train = training_set[:, 0]
y_train = training_set[:, 1]

X_test = data["test_set"]

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

X_train = X_train.astype(np.float32)
X_val   = X_val.astype(np.float32)
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)

y_train = y_train.astype(np.int32)
y_val   = y_val.astype(np.int32)
y_train = np.expand_dims(y_train, axis=-1)
y_val   = np.expand_dims(y_val, axis=-1)

print(f" X_train: {X_train.shape}")
print(f" y_train: {y_train.shape}")
print(f" X_val:   {X_val.shape}")

 X_train: (2092, 64, 128, 1)
 y_train: (2092, 64, 128, 1)
 X_val:   (523, 64, 128, 1)


In [29]:
# def geometric_augmentation(X, y, augmenter, num_augmentations=1):

#     augmented_X, augmented_y = [], []

#     for i in range(len(X)):

#         img = X[i].reshape((1,) + X[i].shape)
#         mask = y[i].reshape((1,) + y[i].shape)

#         for _ in range(num_augmentations):

#             seed = np.random.randint(1e6)

#             img_flow = augmenter.flow(img,   batch_size=1, seed=seed)
#             mask_flow = augmenter.flow(mask, batch_size=1, seed=seed)

#             aug_img  = next(img_flow)[0]
#             aug_mask = next(mask_flow)[0]

#             augmented_X.append(aug_img)
#             augmented_y.append(aug_mask)

#     return np.array(augmented_X), np.array(augmented_y)


In [30]:
# augmenter = ImageDataGenerator(
#     rotation_range = 20,
#     width_shift_range = 0.1,
#     height_shift_range = 0.1,
#     shear_range = 0.1,
#     zoom_range = 0.1,
#     horizontal_flip = True
# )

# X_train, y_train = geometric_augmentation(X_train, y_train, augmenter, num_augmentations = 5)

# print(f" X_train: {X_train.shape}")
# print(f" y_train: {y_train.shape}")

 X_train: (10460, 64, 128, 1)
 y_train: (10460, 64, 128, 1)


In [31]:
# def photometric_augmentation(X, y, augmenter, num_augmentations=1):

#     augmented_X, augmented_y = [], []

#     for i in range(len(X)):
#         img = X[i].reshape((1,) + X[i].shape)
#         label = y[i]

#         for _ in range(num_augmentations):
#             for augmented_img in augmenter.flow(img, batch_size=1):
#                 augmented_X.append(augmented_img[0])
#                 augmented_y.append(label)
#                 break

#     return np.array(augmented_X), np.array(augmented_y)

In [32]:
# augmenter = ImageDataGenerator(
#     brightness_range=[0.2, 1.8],
# )

# X_train, y_train = photometric_augmentation(X_train, y_train, augmenter, num_augmentations = 1)

# print(f" X_train: {X_train.shape}")
# print(f" y_train: {y_train.shape}")

 X_train: (10460, 64, 128, 1)
 y_train: (10460, 64, 128, 1)


In [137]:
class ResidualConnection(Layer):
    def call(self, inputs):

        x1, x2 = inputs

        # gestione mismatch spaziale
        if x1.shape[1] != x2.shape[1] or x1.shape[2] != x2.shape[2]:
            x1 = tf.image.resize(x1, (tf.shape(x2)[1], tf.shape(x2)[2]))

        # gestione del mismatch nei canali
        if x1.shape[-1] < x2.shape[-1]:
            diff = x2.shape[-1] - x1.shape[-1]
            zero_padding = tf.zeros(tf.concat([tf.shape(x1)[:-1], [diff]], axis=0))  # Forma dinamica
            x1 = tf.concat([x1, zero_padding], axis=-1)
        elif x1.shape[-1] > x2.shape[-1]:
            x1 = x1[..., :x2.shape[-1]]



        # Somma residuale
        return x1 + x2


In [132]:
def encoder_block(input_tensor, num_filters, dropout_rate):

    x = layers.Conv2D(num_filters, (3, 3), padding='same', kernel_initializer='he_normal')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(num_filters, (3, 3), padding='same', kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)

    x = ResidualConnection()([input_tensor, x])
    x = layers.Activation('relu')(x)

    skip = x
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(dropout_rate)(x)

    return skip, x


def decoder_block(input_tensor, skip_tensor, num_filters):

    x = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same', kernel_initializer='he_normal')(input_tensor)
    x = layers.concatenate([x, skip_tensor])

    x = layers.Conv2D(num_filters, (3, 3), padding='same', kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)

    x = ResidualConnection()([input_tensor, x])
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(num_filters, (3, 3), padding='same', kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    return x


In [133]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input

def unet_model():

    # input
    inputs = Input(shape=(64, 128, 1))

    # encoder
    skip1, x = encoder_block(inputs, num_filters=64, dropout_rate=0.1)
    skip2, x = encoder_block(x, num_filters=128, dropout_rate=0.2)
    skip3, x = encoder_block(x, num_filters=256, dropout_rate=0.3)
    skip4, x = encoder_block(x, num_filters=512, dropout_rate=0.4)

    # bottleneck
    x = layers.Conv2D(1024, (3, 3), padding='same', kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(1024, (3, 3), padding='same', kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # decoder
    x = decoder_block(x, skip4, num_filters=512)
    x = decoder_block(x, skip3, num_filters=256)
    x = decoder_block(x, skip2, num_filters=128)
    x = decoder_block(x, skip1, num_filters=64)

    # output
    outputs = layers.Conv2D(5, (1, 1), activation='softmax', kernel_initializer='he_normal')(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model

In [138]:
model = unet_model()

In [139]:
class MeanIntersectionOverUnion(tf.keras.metrics.MeanIoU):
    def __init__(self, num_classes, labels_to_exclude=None, name="mean_iou", dtype=None):
        super(MeanIntersectionOverUnion, self).__init__(num_classes=num_classes, name=name, dtype=dtype)
        if labels_to_exclude is None:
            labels_to_exclude = [0]  # Default to excluding label 0
        self.labels_to_exclude = labels_to_exclude

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Convert predictions to class labels
        y_pred = tf.math.argmax(y_pred, axis=-1)

        # Flatten the tensors
        y_true = tf.reshape(y_true, [-1])
        y_pred = tf.reshape(y_pred, [-1])

        # Apply mask to exclude specified labels
        for label in self.labels_to_exclude:
            mask = tf.not_equal(y_true, label)
            y_true = tf.boolean_mask(y_true, mask)
            y_pred = tf.boolean_mask(y_pred, mask)

        # Update the state
        return super().update_state(y_true, y_pred, sample_weight)

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.AdamW(1e-3),
    metrics=["accuracy", MeanIntersectionOverUnion(num_classes=5, labels_to_exclude=[0])]
)

In [140]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=10,
    restore_best_weights=True
)

In [141]:
history = model.fit(
    x = X_train,
    y = y_train,
    epochs = 100,
    callbacks = early_stopping,
    validation_data = (X_val, y_val),
    verbose=1
).history

Epoch 1/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 52s 379ms/step - accuracy: 0.3808 - loss: 1.4492 - mean_iou: 0.1614 - val_accuracy: 0.2051 - val_loss: 85.4971 - val_mean_iou: 0.0541
Epoch 2/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.5227 - loss: 1.1164 - mean_iou: 0.2452 - val_accuracy: 0.4618 - val_loss: 1.8107 - val_mean_iou: 0.2212
Epoch 3/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.5946 - loss: 0.9963 - mean_iou: 0.3084 - val_accuracy: 0.4660 - val_loss: 2.5949 - val_mean_iou: 0.2101
Epoch 4/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.6269 - loss: 0.9247 - mean_iou: 0.3380 - val_accuracy: 0.6333 - val_loss: 0.9505 - val_mean_iou: 0.3424
Epoch 5/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.6418 - loss: 0.8851 - mean_iou: 0.3477 - val_accuracy: 0.6505 - val_loss: 0.8917 - val_mean_iou: 0.3603
Epoch 6/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.6555 - loss: 0.8573 - mean_iou: 0.3599 - val_accuracy: 0.6615 - val_loss: 0.8849 

In [142]:
# Save the trained model to a file with the accuracy included in the filename
model_filename = 'model3.keras'
model.save(model_filename)

timestep_str = datetime.now().strftime("%y%m%d_%H%M%S")
model_filename = f"model_{timestep_str}.keras"
model.save(model_filename)

print(f"Model saved to {model_filename}")

Model saved to model_241208_213105.keras


In [143]:
model = tfk.models.load_model('model3.keras', compile=False)
print(f"Model loaded from {model_filename}")

TypeError: <class 'keras.src.models.functional.Functional'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {'name': 'functional_9', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 64, 128, 1], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_25'}, 'registered_name': None, 'name': 'input_layer_25', 'inbound_nodes': []}, {'module': 'keras.layers', 'class_name': 'Conv2D', 'config': {'name': 'conv2d_307', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'filters': 64, 'kernel_size': [3, 3], 'strides': [1, 1], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': 

In [144]:
preds = model.predict(X_test)
preds = np.argmax(preds, axis=-1)
print(f"Predictions shape: {preds.shape}")

314/314 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step
Predictions shape: (10022, 64, 128)


In [145]:
def y_to_df(y) -> pd.DataFrame:
    """Converts segmentation predictions into a DataFrame format for Kaggle."""
    n_samples = len(y)
    y_flat = y.reshape(n_samples, -1)
    df = pd.DataFrame(y_flat)
    df["id"] = np.arange(n_samples)
    cols = ["id"] + [col for col in df.columns if col != "id"]
    return df[cols]

In [146]:
timestep_str = model_filename.replace("model_", "").replace(".keras", "")
submission_filename = f"submission_{timestep_str}.csv"
submission_df = y_to_df(preds)
submission_df.to_csv(submission_filename, index=False)

from google.colab import files
files.download(submission_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>